In [1]:
# author: Yin 
# 4 27 2020, change panel identifier from id to csid, get identical results as LG
# RRM with panel data, cross elasticities
# Three alternatives: sq,a,b; 
# cross point elasticities for continuous variables in RRM
#
#04 15 test with large mu (mu=10 or mu=1)to see if cross elas estimates are close to RUM results
#04 15 when mu=1, crs elas: rrm/rum are within 78%-135%
#04 15 when mu=0.873, crs elas: rrm/rum are within  

In [2]:
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import *


In [3]:
#Calculating confidence intervals for elasticities requires interval arithmetics 
#pip install python-intervals in anaconda prompt
#in case python-intervals doesn't work, also install a newer version "portion"

import portion as p
import intervals as ivs
import interval as iv

##### print("Running 03logit_rrm_elas_crs_pt_0415.py...")

In [4]:
# Read the data
df = pd.read_csv("agree0623_all_0427.csv")
database = db.Database("agree0623_all_0427",df)
# They are organized as panel data. The variable ID identifies each individual.
database.panel("csid")
# use the names of the variable as Python variable.
globals().update(database.variables)

In [5]:
# Normalize the weights
sumWeight = database.data['weight'].sum()
print(sumWeight) 
##result=7423.735097, sample size=7414

7423.735097


In [6]:
normalizedWeight = weight * 7414 / 7423.735097
# originally weight the whole sample use 1123 cases, 
# then delete nonresponse for choice questions, and 945 cases are used for logit

In [7]:
# Parameters to be estimated 
#NZone variables are already scaled: zone/100

#use 0427 biogeme rrm estimates as starting values 
B_NZONE1 = Beta('B_NZONE1',-0.0456,None,None,0)
B_NZONE2 = Beta('B_NZONE2',-0.0247,None,None,0)
B_NZONE3 = Beta('B_NZONE3',-0.0189,None,None,0)
B_TOWNDOWN = Beta('B_TOWNDOWN',0.0173,None,None,0)
B_TOWNMID = Beta('B_TOWNMID',0.0755,None,None,0)
B_TOWNUP = Beta('B_TOWNUP',0.136,None,None,0)
B_ECO = Beta('B_ECO',0.0354,None,None,0)
B_REC = Beta('B_REC',0.00596,None,None,0)
B_DRY = Beta('B_DRY',-0.142,None,None,0)
B_TAX = Beta('B_TAX',-0.017,None,None,0)

# mu
mu    = Beta('mu',5.51,0,10,0)

In [9]:
# Definition of the utility functions
RSQ = mu * (log( 1 + exp((B_NZONE1/mu) * (nzone1_a-nzone1_sq))) + log(1 + exp((B_NZONE1/mu ) * (nzone1_b-nzone1_sq))) + log(1 + exp((B_NZONE2/mu) * (nzone2_a-nzone2_sq))) + log(1 + exp((B_NZONE2 / mu ) * (nzone2_b-nzone2_sq))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_a-nzone3_sq))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_b-nzone3_sq))) + log( 1 + exp((B_TOWNDOWN / mu ) * (towndown_a-towndown_sq))) + log(1 + exp((B_TOWNDOWN / mu ) * (towndown_b-towndown_sq)))  + log(1 + exp((B_TOWNMID / mu ) * (townmid_a-townmid_sq))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_b-townmid_sq))) + log(1 + exp((B_TOWNUP / mu ) * (townup_a-townup_sq))) + log(1 + exp((B_TOWNUP / mu ) * (townup_b-townup_sq))) + log( 1 + exp((B_ECO / mu ) * (eco_a-eco_sq))) + log(1 + exp((B_ECO / mu ) * (eco_b-eco_sq))) + log(1 + exp((B_REC / mu ) * (rec_a-rec_sq))) + log(1 + exp((B_REC / mu ) * (rec_b-rec_sq))) + log(1 + exp((B_DRY / mu ) * (dry_a-dry_sq))) + log(1 + exp((B_DRY / mu ) * (dry_b-dry_sq))) + log(1 + exp((B_TAX / mu ) * (tax_a-tax_sq))) + log(1 + exp((B_TAX / mu ) * (tax_b-tax_sq))))                                                
          
RA  = mu * (log( 1 + exp((B_NZONE1/mu) * (nzone1_sq-nzone1_a))) + log(1 + exp((B_NZONE1/mu ) * (nzone1_b-nzone1_a))) + log(1 + exp((B_NZONE2/mu) * (nzone2_sq-nzone2_a))) + log(1 + exp((B_NZONE2 / mu ) * (nzone2_b-nzone2_a))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_sq-nzone3_a))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_b-nzone3_a))) + log( 1 + exp((B_TOWNDOWN / mu ) * (towndown_sq-towndown_a))) + log(1 + exp((B_TOWNDOWN / mu ) * (towndown_b-towndown_a))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_sq-townmid_a))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_b-townmid_a))) + log(1 + exp((B_TOWNUP / mu ) * (townup_sq-townup_a))) + log(1 + exp((B_TOWNUP / mu ) * (townup_b-townup_a))) + log( 1 + exp((B_ECO / mu ) * (eco_sq-eco_a))) + log(1 + exp((B_ECO / mu ) * (eco_b-eco_a))) + log(1 + exp((B_REC / mu ) * (rec_sq-rec_a))) + log(1 + exp((B_REC / mu ) * (rec_b-rec_a))) + log(1 + exp((B_DRY / mu ) * (dry_sq-dry_a))) + log(1 + exp((B_DRY / mu ) * (dry_b-dry_a))) + log(1 + exp((B_TAX / mu ) * (tax_sq-tax_a))) + log(1 + exp((B_TAX / mu ) * (tax_b-tax_a))))   
           
RB  = mu * (log( 1 + exp((B_NZONE1/mu) * (nzone1_sq-nzone1_b))) + log(1 + exp((B_NZONE1/mu ) * (nzone1_a-nzone1_b))) + log(1 + exp((B_NZONE2/mu) * (nzone2_sq-nzone2_b))) + log(1 + exp((B_NZONE2 / mu ) * (nzone2_a-nzone2_b))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_sq-nzone3_b))) + log(1 + exp((B_NZONE3 / mu ) * (nzone3_a-nzone3_b))) + log( 1 + exp((B_TOWNDOWN / mu ) * (towndown_sq-towndown_b))) + log(1 + exp((B_TOWNDOWN / mu ) * (towndown_a-towndown_b))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_sq-townmid_b))) + log(1 + exp((B_TOWNMID / mu ) * (townmid_a-townmid_b))) + log(1 + exp((B_TOWNUP / mu ) * (townup_sq-townup_b))) + log(1 + exp((B_TOWNUP / mu ) * (townup_a-townup_b))) + log( 1 + exp((B_ECO / mu ) * (eco_sq-eco_b))) + log(1 + exp((B_ECO / mu ) * (eco_a-eco_b))) + log(1 + exp((B_REC / mu ) * (rec_sq-rec_b))) + log(1 + exp((B_REC / mu ) * (rec_a-rec_b))) + log(1 + exp((B_DRY / mu ) * (dry_sq-dry_b))) + log(1 + exp((B_DRY / mu ) * (dry_a-dry_b))) + log(1 + exp((B_TAX / mu ) * (tax_sq-tax_b))) + log(1 + exp((B_TAX / mu ) * (tax_a-tax_b))))   

In [10]:
# Associate utility functions with the numbering of alternatives
V = {1: -RSQ,
     2: -RA,
     3: -RB}

# Associate the availability conditions with the alternatives
# all alternatives are available for each individual
av = {1: 1,
      2: 1,
      3: 1}

In [11]:
# Definition of the model. This is the contribution of each observation to the log likelihood function.
prob_sq = models.logit(V,av,1)
prob_a  = models.logit(V,av,2)
prob_b  = models.logit(V,av,3)


In [12]:
# Calculation of the cross elasticities. 
# q(sq,a)= exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a))/(1+exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a)))
# 
# alt_sq wrt attri in alt a
cross_elas_sq_nzone1_a=(B_NZONE1) * nzone1_a*(-prob_a+ prob_b* (exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a))/(1+exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a))))-(prob_a +prob_b)*(exp((B_NZONE1/mu)*(nzone1_b -nzone1_a))/(1+exp((B_NZONE1/mu)*(nzone1_b -nzone1_a)))))
cross_elas_sq_nzone2_a=(B_NZONE2) * nzone2_a*(-prob_a+ prob_b* (exp((B_NZONE2/mu)*(nzone2_sq -nzone2_a))/(1+exp((B_NZONE2/mu)*(nzone2_sq -nzone2_a))))-(prob_a +prob_b)*(exp((B_NZONE2/mu)*(nzone2_b -nzone2_a))/(1+exp((B_NZONE2/mu)*(nzone2_b -nzone2_a)))))
cross_elas_sq_nzone3_a=(B_NZONE3) * nzone3_a*(-prob_a+ prob_b* (exp((B_NZONE3/mu)*(nzone3_sq -nzone3_a))/(1+exp((B_NZONE3/mu)*(nzone3_sq -nzone3_a))))-(prob_a +prob_b)*(exp((B_NZONE3/mu)*(nzone3_b -nzone3_a))/(1+exp((B_NZONE3/mu)*(nzone3_b -nzone3_a)))))

cross_elas_sq_eco_a=(B_ECO) * eco_a*(-prob_a+ prob_b* (exp((B_ECO/mu)*(eco_sq -eco_a))/(1+exp((B_ECO/mu)*(eco_sq -eco_a))))-(prob_a +prob_b)*(exp((B_ECO/mu)*(eco_b -eco_a))/(1+exp((B_ECO/mu)*(eco_b -eco_a)))))
cross_elas_sq_rec_a=(B_REC) * rec_a*(-prob_a+ prob_b* (exp((B_REC/mu)*(rec_sq -rec_a))/(1+exp((B_REC/mu)*(rec_sq -rec_a))))-(prob_a +prob_b)*(exp((B_REC/mu)*(rec_b -rec_a))/(1+exp((B_REC/mu)*(rec_b -rec_a)))))
cross_elas_sq_dry_a=(B_DRY) * dry_a*(-prob_a+ prob_b* (exp((B_DRY/mu)*(dry_sq -dry_a))/(1+exp((B_DRY/mu)*(dry_sq -dry_a))))-(prob_a +prob_b)*(exp((B_DRY/mu)*(dry_b -dry_a))/(1+exp((B_DRY/mu)*(dry_b -dry_a)))))
cross_elas_sq_tax_a=(B_TAX) * tax_a*(-prob_a+ prob_b* (exp((B_TAX/mu)*(tax_sq -tax_a))/(1+exp((B_TAX/mu)*(tax_sq -tax_a))))-(prob_a +prob_b)*(exp((B_TAX/mu)*(tax_b -tax_a))/(1+exp((B_TAX/mu)*(tax_b -tax_a)))))

# alt_sq wrt attri in alt b
cross_elas_sq_nzone1_b=(B_NZONE1) * nzone1_b*(-prob_b+ prob_a* (exp((B_NZONE1/mu)*(nzone1_sq -nzone1_b))/(1+exp((B_NZONE1/mu)*(nzone1_sq -nzone1_b))))-(prob_b +prob_a)*(exp((B_NZONE1/mu)*(nzone1_a -nzone1_b))/(1+exp((B_NZONE1/mu)*(nzone1_a -nzone1_b)))))
cross_elas_sq_nzone2_b=(B_NZONE2) * nzone2_b*(-prob_b+ prob_a* (exp((B_NZONE2/mu)*(nzone2_sq -nzone2_b))/(1+exp((B_NZONE2/mu)*(nzone2_sq -nzone2_b))))-(prob_b +prob_a)*(exp((B_NZONE2/mu)*(nzone2_a -nzone2_b))/(1+exp((B_NZONE2/mu)*(nzone2_a -nzone2_b)))))
cross_elas_sq_nzone3_b=(B_NZONE3) * nzone3_b*(-prob_b+ prob_a* (exp((B_NZONE3/mu)*(nzone3_sq -nzone3_b))/(1+exp((B_NZONE3/mu)*(nzone3_sq -nzone3_b))))-(prob_b +prob_a)*(exp((B_NZONE3/mu)*(nzone3_a -nzone3_b))/(1+exp((B_NZONE3/mu)*(nzone3_a -nzone3_b)))))

cross_elas_sq_eco_b=(B_ECO) * eco_b*(-prob_b+ prob_a* (exp((B_ECO/mu)*(eco_sq -eco_b))/(1+exp((B_ECO/mu)*(eco_sq -eco_b))))-(prob_b +prob_a)*(exp((B_ECO/mu)*(eco_a -eco_b))/(1+exp((B_ECO/mu)*(eco_a -eco_b)))))
cross_elas_sq_rec_b=(B_REC) * rec_b*(-prob_b+ prob_a* (exp((B_REC/mu)*(rec_sq -rec_b))/(1+exp((B_REC/mu)*(rec_sq -rec_b))))-(prob_b +prob_a)*(exp((B_REC/mu)*(rec_a -rec_b))/(1+exp((B_REC/mu)*(rec_a -rec_b)))))
cross_elas_sq_dry_b=(B_DRY) * dry_b*(-prob_b+ prob_a* (exp((B_DRY/mu)*(dry_sq -dry_b))/(1+exp((B_DRY/mu)*(dry_sq -dry_b))))-(prob_b +prob_a)*(exp((B_DRY/mu)*(dry_a -dry_b))/(1+exp((B_DRY/mu)*(dry_a -dry_b)))))
cross_elas_sq_tax_b=(B_TAX) * tax_b*(-prob_b+ prob_a* (exp((B_TAX/mu)*(tax_sq -tax_b))/(1+exp((B_TAX/mu)*(tax_sq -tax_b))))-(prob_b +prob_a)*(exp((B_TAX/mu)*(tax_a -tax_b))/(1+exp((B_TAX/mu)*(tax_a -tax_b)))))


In [13]:
# Calculation of the cross elasticities. 
# q(sq,a)= exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a))/(1+exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a)))
# 
# alt_a wrt attri in alt sq
cross_elas_a_nzone1_sq=(B_NZONE1) * nzone1_sq*(-prob_sq+ prob_b* (exp((B_NZONE1/mu)*(nzone1_a -nzone1_sq))/(1+exp((B_NZONE1/mu)*(nzone1_a -nzone1_sq))))-(prob_sq +prob_b)*(exp((B_NZONE1/mu)*(nzone1_b -nzone1_sq))/(1+exp((B_NZONE1/mu)*(nzone1_b -nzone1_sq)))))
cross_elas_a_nzone2_sq=(B_NZONE2) * nzone2_sq*(-prob_sq+ prob_b* (exp((B_NZONE2/mu)*(nzone2_a -nzone2_sq))/(1+exp((B_NZONE2/mu)*(nzone2_a -nzone2_sq))))-(prob_sq +prob_b)*(exp((B_NZONE2/mu)*(nzone2_b -nzone2_sq))/(1+exp((B_NZONE2/mu)*(nzone2_b -nzone2_sq)))))
cross_elas_a_nzone3_sq=(B_NZONE3) * nzone3_sq*(-prob_sq+ prob_b* (exp((B_NZONE3/mu)*(nzone3_a -nzone3_sq))/(1+exp((B_NZONE3/mu)*(nzone3_a -nzone3_sq))))-(prob_sq +prob_b)*(exp((B_NZONE3/mu)*(nzone3_b -nzone3_sq))/(1+exp((B_NZONE3/mu)*(nzone3_b -nzone3_sq)))))

cross_elas_a_eco_sq=(B_ECO) * eco_sq*(-prob_sq+ prob_b* (exp((B_ECO/mu)*(eco_a -eco_sq))/(1+exp((B_ECO/mu)*(eco_a -eco_sq))))-(prob_sq +prob_b)*(exp((B_ECO/mu)*(eco_b -eco_sq))/(1+exp((B_ECO/mu)*(eco_b -eco_sq)))))
cross_elas_a_rec_sq=(B_REC) * rec_sq*(-prob_sq+ prob_b* (exp((B_REC/mu)*(rec_a -rec_sq))/(1+exp((B_REC/mu)*(rec_a -rec_sq))))-(prob_sq +prob_b)*(exp((B_REC/mu)*(rec_b -rec_sq))/(1+exp((B_REC/mu)*(rec_b -rec_sq)))))
cross_elas_a_dry_sq=(B_DRY) * dry_sq*(-prob_sq+ prob_b* (exp((B_DRY/mu)*(dry_a -dry_sq))/(1+exp((B_DRY/mu)*(dry_a -dry_sq))))-(prob_sq +prob_b)*(exp((B_DRY/mu)*(dry_b -dry_sq))/(1+exp((B_DRY/mu)*(dry_b -dry_sq)))))
cross_elas_a_tax_sq=(B_TAX) * tax_sq*(-prob_sq+ prob_b* (exp((B_TAX/mu)*(tax_a -tax_sq))/(1+exp((B_TAX/mu)*(tax_a -tax_sq))))-(prob_sq +prob_b)*(exp((B_TAX/mu)*(tax_b -tax_sq))/(1+exp((B_TAX/mu)*(tax_b -tax_sq)))))

# alt_a wrt attri in alt b
cross_elas_a_nzone1_b=(B_NZONE1) * nzone1_b*(-prob_b+ prob_sq* (exp((B_NZONE1/mu)*(nzone1_a -nzone1_b))/(1+exp((B_NZONE1/mu)*(nzone1_a -nzone1_b))))-(prob_sq +prob_b)*(exp((B_NZONE1/mu)*(nzone1_sq -nzone1_b))/(1+exp((B_NZONE1/mu)*(nzone1_sq -nzone1_b)))))
cross_elas_a_nzone2_b=(B_NZONE2) * nzone2_b*(-prob_b+ prob_sq* (exp((B_NZONE2/mu)*(nzone2_a -nzone2_b))/(1+exp((B_NZONE2/mu)*(nzone2_a -nzone2_b))))-(prob_sq +prob_b)*(exp((B_NZONE2/mu)*(nzone2_sq -nzone2_b))/(1+exp((B_NZONE2/mu)*(nzone2_sq -nzone2_b)))))
cross_elas_a_nzone3_b=(B_NZONE3) * nzone3_b*(-prob_b+ prob_sq* (exp((B_NZONE3/mu)*(nzone3_a -nzone3_b))/(1+exp((B_NZONE3/mu)*(nzone3_a -nzone3_b))))-(prob_sq +prob_b)*(exp((B_NZONE3/mu)*(nzone3_sq -nzone3_b))/(1+exp((B_NZONE3/mu)*(nzone3_sq -nzone3_b)))))

cross_elas_a_eco_b=(B_ECO) * eco_b*(-prob_b+ prob_sq* (exp((B_ECO/mu)*(eco_a -eco_b))/(1+exp((B_ECO/mu)*(eco_a -eco_b))))-(prob_sq +prob_b)*(exp((B_ECO/mu)*(eco_sq -eco_b))/(1+exp((B_ECO/mu)*(eco_sq -eco_b)))))
cross_elas_a_rec_b=(B_REC) * rec_b*(-prob_b+ prob_sq* (exp((B_REC/mu)*(rec_a -rec_b))/(1+exp((B_REC/mu)*(rec_a -rec_b))))-(prob_sq +prob_b)*(exp((B_REC/mu)*(rec_sq -rec_b))/(1+exp((B_REC/mu)*(rec_sq -rec_b)))))
cross_elas_a_dry_b=(B_DRY) * dry_b*(-prob_b+ prob_sq* (exp((B_DRY/mu)*(dry_a -dry_b))/(1+exp((B_DRY/mu)*(dry_a -dry_b))))-(prob_sq +prob_b)*(exp((B_DRY/mu)*(dry_sq -dry_b))/(1+exp((B_DRY/mu)*(dry_sq -dry_b)))))
cross_elas_a_tax_b=(B_TAX) * tax_b*(-prob_b+ prob_sq* (exp((B_TAX/mu)*(tax_a -tax_b))/(1+exp((B_TAX/mu)*(tax_a -tax_b))))-(prob_sq +prob_b)*(exp((B_TAX/mu)*(tax_sq -tax_b))/(1+exp((B_TAX/mu)*(tax_sq -tax_b)))))


In [14]:
# Calculation of the cross elasticities. 
# q(sq,a)= exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a))/(1+exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a)))
# 
# alt_b wrt attri in alt sq
cross_elas_b_nzone1_sq=(B_NZONE1) * nzone1_sq*(-prob_sq+ prob_a* (exp((B_NZONE1/mu)*(nzone1_b -nzone1_sq))/(1+exp((B_NZONE1/mu)*(nzone1_b -nzone1_sq))))-(prob_sq +prob_a)*(exp((B_NZONE1/mu)*(nzone1_a -nzone1_sq))/(1+exp((B_NZONE1/mu)*(nzone1_a -nzone1_sq)))))
cross_elas_b_nzone2_sq=(B_NZONE2) * nzone2_sq*(-prob_sq+ prob_a* (exp((B_NZONE2/mu)*(nzone2_b -nzone2_sq))/(1+exp((B_NZONE2/mu)*(nzone2_b -nzone2_sq))))-(prob_sq +prob_a)*(exp((B_NZONE2/mu)*(nzone2_a -nzone2_sq))/(1+exp((B_NZONE2/mu)*(nzone2_a -nzone2_sq)))))
cross_elas_b_nzone3_sq=(B_NZONE3) * nzone3_sq*(-prob_sq+ prob_a* (exp((B_NZONE3/mu)*(nzone3_b -nzone3_sq))/(1+exp((B_NZONE3/mu)*(nzone3_b -nzone3_sq))))-(prob_sq +prob_a)*(exp((B_NZONE3/mu)*(nzone3_a -nzone3_sq))/(1+exp((B_NZONE3/mu)*(nzone3_a -nzone3_sq)))))

cross_elas_b_eco_sq=(B_ECO) * eco_sq*(-prob_sq+ prob_a* (exp((B_ECO/mu)*(eco_b -eco_sq))/(1+exp((B_ECO/mu)*(eco_b -eco_sq))))-(prob_sq +prob_a)*(exp((B_ECO/mu)*(eco_a -eco_sq))/(1+exp((B_ECO/mu)*(eco_a -eco_sq)))))
cross_elas_b_rec_sq=(B_REC) * rec_sq*(-prob_sq+ prob_a* (exp((B_REC/mu)*(rec_b -rec_sq))/(1+exp((B_REC/mu)*(rec_b -rec_sq))))-(prob_sq +prob_a)*(exp((B_REC/mu)*(rec_a -rec_sq))/(1+exp((B_REC/mu)*(rec_a -rec_sq)))))
cross_elas_b_dry_sq=(B_DRY) * dry_sq*(-prob_sq+ prob_a* (exp((B_DRY/mu)*(dry_b -dry_sq))/(1+exp((B_DRY/mu)*(dry_b -dry_sq))))-(prob_sq +prob_a)*(exp((B_DRY/mu)*(dry_a -dry_sq))/(1+exp((B_DRY/mu)*(dry_a -dry_sq)))))
cross_elas_b_tax_sq=(B_TAX) * tax_sq*(-prob_sq+ prob_a* (exp((B_TAX/mu)*(tax_b -tax_sq))/(1+exp((B_TAX/mu)*(tax_b -tax_sq))))-(prob_sq +prob_a)*(exp((B_TAX/mu)*(tax_a -tax_sq))/(1+exp((B_TAX/mu)*(tax_a -tax_sq)))))

# alt_b wrt attri in alt a
cross_elas_b_nzone1_a=(B_NZONE1) * nzone1_a*(-prob_a+ prob_sq* (exp((B_NZONE1/mu)*(nzone1_b -nzone1_a))/(1+exp((B_NZONE1/mu)*(nzone1_b -nzone1_a))))-(prob_sq +prob_a)*(exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a))/(1+exp((B_NZONE1/mu)*(nzone1_sq -nzone1_a)))))
cross_elas_b_nzone2_a=(B_NZONE2) * nzone2_a*(-prob_a+ prob_sq* (exp((B_NZONE2/mu)*(nzone2_b -nzone2_a))/(1+exp((B_NZONE2/mu)*(nzone2_b -nzone2_a))))-(prob_sq +prob_a)*(exp((B_NZONE2/mu)*(nzone2_sq -nzone2_a))/(1+exp((B_NZONE2/mu)*(nzone2_sq -nzone2_a)))))
cross_elas_b_nzone3_a=(B_NZONE3) * nzone3_a*(-prob_a+ prob_sq* (exp((B_NZONE3/mu)*(nzone3_b -nzone3_a))/(1+exp((B_NZONE3/mu)*(nzone3_b -nzone3_a))))-(prob_sq +prob_a)*(exp((B_NZONE3/mu)*(nzone3_sq -nzone3_a))/(1+exp((B_NZONE3/mu)*(nzone3_sq -nzone3_a)))))

cross_elas_b_eco_a=(B_ECO) * eco_a*(-prob_a+ prob_sq* (exp((B_ECO/mu)*(eco_b -eco_a))/(1+exp((B_ECO/mu)*(eco_b -eco_a))))-(prob_sq +prob_a)*(exp((B_ECO/mu)*(eco_sq -eco_a))/(1+exp((B_ECO/mu)*(eco_sq -eco_a)))))
cross_elas_b_rec_a=(B_REC) * rec_a*(-prob_a+ prob_sq* (exp((B_REC/mu)*(rec_b -rec_a))/(1+exp((B_REC/mu)*(rec_b -rec_a))))-(prob_sq +prob_a)*(exp((B_REC/mu)*(rec_sq -rec_a))/(1+exp((B_REC/mu)*(rec_sq -rec_a)))))
cross_elas_b_dry_a=(B_DRY) * dry_a*(-prob_a+ prob_sq* (exp((B_DRY/mu)*(dry_b -dry_a))/(1+exp((B_DRY/mu)*(dry_b -dry_a))))-(prob_sq +prob_a)*(exp((B_DRY/mu)*(dry_sq -dry_a))/(1+exp((B_DRY/mu)*(dry_sq -dry_a)))))
cross_elas_b_tax_a=(B_TAX) * tax_a*(-prob_a+ prob_sq* (exp((B_TAX/mu)*(tax_b -tax_a))/(1+exp((B_TAX/mu)*(tax_b -tax_a))))-(prob_sq +prob_a)*(exp((B_TAX/mu)*(tax_sq -tax_a))/(1+exp((B_TAX/mu)*(tax_sq -tax_a)))))


In [15]:
simulate = {'nweight': normalizedWeight,
            'Prob. No Action': prob_sq,
            'Prob. Plan A': prob_a,
            'Prob. Plan B':prob_b,
            'cross_elas_sq_nzone1_a':cross_elas_sq_nzone1_a,
            'cross_elas_sq_nzone2_a':cross_elas_sq_nzone2_a,
            'cross_elas_sq_nzone3_a':cross_elas_sq_nzone3_a,
            'cross_elas_sq_eco_a':cross_elas_sq_eco_a,
            'cross_elas_sq_rec_a':cross_elas_sq_rec_a,
            'cross_elas_sq_dry_a':cross_elas_sq_dry_a,
            'cross_elas_sq_tax_a':cross_elas_sq_tax_a,
            'cross_elas_sq_nzone1_b':cross_elas_sq_nzone1_b,
            'cross_elas_sq_nzone2_b':cross_elas_sq_nzone2_b,
            'cross_elas_sq_nzone3_b':cross_elas_sq_nzone3_b,
            'cross_elas_sq_eco_b':cross_elas_sq_eco_b,
            'cross_elas_sq_rec_b':cross_elas_sq_rec_b,
            'cross_elas_sq_dry_b':cross_elas_sq_dry_b,
            'cross_elas_sq_tax_b':cross_elas_sq_tax_b,          
            'cross_elas_a_nzone1_sq':cross_elas_a_nzone1_sq,
            'cross_elas_a_nzone2_sq':cross_elas_a_nzone2_sq,
            'cross_elas_a_nzone3_sq':cross_elas_a_nzone3_sq,
            'cross_elas_a_eco_sq':cross_elas_a_eco_sq,
            'cross_elas_a_rec_sq':cross_elas_a_rec_sq,
            'cross_elas_a_dry_sq':cross_elas_a_dry_sq,
            'cross_elas_a_tax_sq':cross_elas_a_tax_sq,
            'cross_elas_a_nzone1_b':cross_elas_a_nzone1_b,
            'cross_elas_a_nzone2_b':cross_elas_a_nzone2_b,
            'cross_elas_a_nzone3_b':cross_elas_a_nzone3_b,
            'cross_elas_a_eco_b':cross_elas_a_eco_b,
            'cross_elas_a_rec_b':cross_elas_a_rec_b,
            'cross_elas_a_dry_b':cross_elas_a_dry_b,
            'cross_elas_a_tax_b':cross_elas_a_tax_b,           
            'cross_elas_b_nzone1_sq':cross_elas_b_nzone1_sq,
            'cross_elas_b_nzone2_sq':cross_elas_b_nzone2_sq,
            'cross_elas_b_nzone3_sq':cross_elas_b_nzone3_sq,
            'cross_elas_b_eco_sq':cross_elas_b_eco_sq,
            'cross_elas_b_rec_sq':cross_elas_b_rec_sq,
            'cross_elas_b_dry_sq':cross_elas_b_dry_sq,
            'cross_elas_b_tax_sq':cross_elas_b_tax_sq,
            'cross_elas_b_nzone1_a':cross_elas_b_nzone1_a,
            'cross_elas_b_nzone2_a':cross_elas_b_nzone2_a,
            'cross_elas_b_nzone3_a':cross_elas_b_nzone3_a,
            'cross_elas_b_eco_a':cross_elas_b_eco_a,
            'cross_elas_b_rec_a':cross_elas_b_rec_a,
            'cross_elas_b_dry_a':cross_elas_b_dry_a,
            'cross_elas_b_tax_a':cross_elas_b_tax_a}


In [16]:
biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "03logit_rrm_elas_crs_pt_0429"

In [17]:
# Retrieve the values of the parameters
# First, extract the names of parameters needed for the simulation
betas = biogeme.freeBetaNames

# Read the estimation results from the file
results = res.bioResults(pickleFile='03logit_rrm_0428.pickle')

# Extract the values that are necessary
betaValues = results.getBetaValues(betas)

# simulatedValues is a Panda dataframe with the same number of rows as the database, and as many columns as formulas to simulate.
# simulate the formulas using the nomial values
simulatedValues = biogeme.simulate(betaValues)

In [18]:
# We calculate the weighted probabilitie
simulatedValues['Weighted Prob. No Action'] = simulatedValues['nweight'] * simulatedValues['Prob. No Action']

simulatedValues['Weighted Prob. Plan A'] = simulatedValues['nweight'] * simulatedValues['Prob. Plan A']

simulatedValues['Weighted Prob. Plan B'] = simulatedValues['nweight'] * simulatedValues['Prob. Plan B']

In [19]:
denominator_sq = simulatedValues['Weighted Prob. No Action'].sum()
denominator_a  = simulatedValues['Weighted Prob. Plan A'].sum()
denominator_b  = simulatedValues['Weighted Prob. Plan B'].sum()

In [20]:
# aggregate cross elasticities_sq ###############################################################
# function=sum(E*Weighted prob/sum(weighted prob))
# aggregate elas _sq to attri in alt a####################################################
cross_elas_term_sq_nzone1_a = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_nzone1_a'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt nzone1_a: {cross_elas_term_sq_nzone1_a:.3g}")
cross_elas_term_sq_nzone2_a = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_nzone2_a'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt nzone2_a: {cross_elas_term_sq_nzone2_a:.3g}")
cross_elas_term_sq_nzone3_a = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_nzone3_a'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt nzone3_a: {cross_elas_term_sq_nzone3_a:.3g}")

cross_elas_term_sq_eco_a = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_eco_a'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt eco_a: {cross_elas_term_sq_eco_a:.3g}")
cross_elas_term_sq_rec_a = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_rec_a'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt rec_a: {cross_elas_term_sq_rec_a:.3g}")
cross_elas_term_sq_dry_a = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_dry_a'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt dry_a: {cross_elas_term_sq_dry_a:.3g}")

cross_elas_term_sq_tax_a = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_tax_a'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt tax_a: {cross_elas_term_sq_tax_a:.3g}")

# aggregate elas _sq to attri in alt b #################################################################
            
cross_elas_term_sq_nzone1_b = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_nzone1_b'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt nzone1_b: {cross_elas_term_sq_nzone1_b:.3g}")
cross_elas_term_sq_nzone2_b = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_nzone2_b'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt nzone2_b: {cross_elas_term_sq_nzone2_b:.3g}")
cross_elas_term_sq_nzone3_b = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_nzone3_b'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt nzone3_b: {cross_elas_term_sq_nzone3_b:.3g}")

cross_elas_term_sq_eco_b = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_eco_b'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt eco_b: {cross_elas_term_sq_eco_b:.3g}")
cross_elas_term_sq_rec_b = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_rec_b'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt rec_b: {cross_elas_term_sq_rec_b:.3g}")
cross_elas_term_sq_dry_b = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_dry_b'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt dry_b: {cross_elas_term_sq_dry_b:.3g}")

cross_elas_term_sq_tax_b = (simulatedValues['Weighted Prob. No Action']
  * simulatedValues['cross_elas_sq_tax_b'] / denominator_sq).sum()
print(f"Aggregate cross elasticity of sq wrt tax_b: {cross_elas_term_sq_tax_b:.3g}")

# aggregate elasticities_a wrt attri in alt sq  #################################################################
cross_elas_term_a_nzone1_sq = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_nzone1_sq'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt nzone1_sq: {cross_elas_term_a_nzone1_sq:.3g}")
cross_elas_term_a_nzone2_sq = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_nzone2_sq'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt nzone2_sq: {cross_elas_term_a_nzone2_sq:.3g}")
cross_elas_term_a_nzone3_sq = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_nzone3_sq'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt nzone3_sq: {cross_elas_term_a_nzone3_sq:.3g}")

cross_elas_term_a_eco_sq = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_eco_sq'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt eco_sq: {cross_elas_term_a_eco_sq:.3g}")
cross_elas_term_a_rec_sq = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_rec_sq'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt rec_sq: {cross_elas_term_a_rec_sq:.3g}")
cross_elas_term_a_dry_sq = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_dry_sq'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt dry_sq: {cross_elas_term_a_dry_sq:.3g}")

cross_elas_term_a_tax_sq = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_tax_sq'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt tax_sq: {cross_elas_term_a_tax_sq:.3g}")

# aggregate elasticities_a wrt attri in alt b ###############################################################
cross_elas_term_a_nzone1_b = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_nzone1_b'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt nzone1_b: {cross_elas_term_a_nzone1_b:.3g}")
cross_elas_term_a_nzone2_b = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_nzone2_b'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt nzone2_b: {cross_elas_term_a_nzone2_b:.3g}")
cross_elas_term_a_nzone3_b = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_nzone3_b'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt nzone3_b: {cross_elas_term_a_nzone3_b:.3g}")

cross_elas_term_a_eco_b = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_eco_b'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt eco_b: {cross_elas_term_a_eco_b:.3g}")
cross_elas_term_a_rec_b = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_rec_b'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt rec_b: {cross_elas_term_a_rec_b:.3g}")
cross_elas_term_a_dry_b = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_dry_b'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt dry_b: {cross_elas_term_a_dry_b:.3g}")

cross_elas_term_a_tax_b = (simulatedValues['Weighted Prob. Plan A']
  * simulatedValues['cross_elas_a_tax_b'] / denominator_a).sum()
print(f"Aggregate cross elasticity of a wrt tax_b: {cross_elas_term_a_tax_b:.3g}")

# aggregate elasticities_b wrt attri in alt sq ###############################################################
cross_elas_term_b_nzone1_sq = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_nzone1_sq'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt nzone1_sq: {cross_elas_term_b_nzone1_sq:.3g}")
cross_elas_term_b_nzone2_sq = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_nzone2_sq'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt nzone2_sq: {cross_elas_term_b_nzone2_sq:.3g}")
cross_elas_term_b_nzone3_sq = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_nzone3_sq'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt nzone3_sq: {cross_elas_term_b_nzone3_sq:.3g}")

cross_elas_term_b_eco_sq = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_eco_sq'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt eco_sq: {cross_elas_term_b_eco_sq:.3g}")
cross_elas_term_b_rec_sq = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_rec_sq'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt rec_sq: {cross_elas_term_b_rec_sq:.3g}")
cross_elas_term_b_dry_sq = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_dry_sq'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt dry_sq: {cross_elas_term_b_dry_sq:.3g}")

cross_elas_term_b_tax_sq = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_tax_sq'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt tax_sq: {cross_elas_term_b_tax_sq:.3g}")


# aggregate elasticities_b wrt attri in alt a ###################################
cross_elas_term_b_nzone1_a = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_nzone1_a'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt nzone1_a: {cross_elas_term_b_nzone1_a:.3g}")
cross_elas_term_b_nzone2_a = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_nzone2_a'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt nzone2_a: {cross_elas_term_b_nzone2_a:.3g}")
cross_elas_term_b_nzone3_a = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_nzone3_a'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt nzone3_a: {cross_elas_term_b_nzone3_a:.3g}")

cross_elas_term_b_eco_a = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_eco_a'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt eco_a: {cross_elas_term_b_eco_a:.3g}")
cross_elas_term_b_rec_a = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_rec_a'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt rec_a: {cross_elas_term_b_rec_a:.3g}")
cross_elas_term_b_dry_a = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_dry_a'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt dry_a: {cross_elas_term_b_dry_a:.3g}")

cross_elas_term_b_tax_a = (simulatedValues['Weighted Prob. Plan B']
  * simulatedValues['cross_elas_b_tax_a'] / denominator_b).sum()
print(f"Aggregate cross elasticity of b wrt tax_a: {cross_elas_term_b_tax_a:.3g}")


Aggregate cross elasticity of sq wrt nzone1_a: 0.251
Aggregate cross elasticity of sq wrt nzone2_a: 0.177
Aggregate cross elasticity of sq wrt nzone3_a: 0.231
Aggregate cross elasticity of sq wrt eco_a: -0.749
Aggregate cross elasticity of sq wrt rec_a: -0.244
Aggregate cross elasticity of sq wrt dry_a: 0.121
Aggregate cross elasticity of sq wrt tax_a: 0.159
Aggregate cross elasticity of sq wrt nzone1_b: 0.276
Aggregate cross elasticity of sq wrt nzone2_b: 0.177
Aggregate cross elasticity of sq wrt nzone3_b: 0.227
Aggregate cross elasticity of sq wrt eco_b: -0.739
Aggregate cross elasticity of sq wrt rec_b: -0.211
Aggregate cross elasticity of sq wrt dry_b: 0.117
Aggregate cross elasticity of sq wrt tax_b: 0.127
Aggregate cross elasticity of a wrt nzone1_sq: 0.191
Aggregate cross elasticity of a wrt nzone2_sq: 0.092
Aggregate cross elasticity of a wrt nzone3_sq: 0.108
Aggregate cross elasticity of a wrt eco_sq: -0.304
Aggregate cross elasticity of a wrt rec_sq: -0.0723
Aggregate cross 

In [21]:
biogeme.createLogFile()


# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

biogeme.modelName = "03logit_rrm_elas_crs_pt_0429"


print("Results=",results)

               Value    Std err     t-test       p-value  Rob. Std err  \
B_DRY      -0.142453   0.018803  -7.576220  3.552714e-14      0.018724   
B_ECO       0.035448   0.002045  17.333557  0.000000e+00      0.001989   
B_NZONE1   -0.045550   0.004159 -10.951057  0.000000e+00      0.004167   
B_NZONE2   -0.024696   0.005351  -4.615089  3.929261e-06      0.005344   
B_NZONE3   -0.018939   0.005494  -3.447555  5.656851e-04      0.005492   
B_REC       0.005956   0.000529  11.259845  0.000000e+00      0.000528   
B_TAX      -0.017001   0.001638 -10.379557  0.000000e+00      0.001550   
B_TOWNDOWN  0.017289   0.027762   0.622748  5.334501e-01      0.027721   
B_TOWNMID   0.075533   0.027023   2.795177  5.187133e-03      0.026814   
B_TOWNUP    0.136175   0.029943   4.547752  5.422189e-06      0.029674   
mu          5.514560  41.906038   0.131593  8.953059e-01     38.016470   

            Rob. t-test  Rob. p-value  
B_DRY         -7.608024  2.775558e-14  
B_ECO         17.824216  0.0000